---
title: Building a Company Brochure with AI
author: Sadashiv
execute:
  echo: true  # Optional: prevents code from being printed in the output
  eval: false  # Prevents the notebook from being re-executed during render
  freeze: auto  # Preserves existing outputs if the notebook has already been executed
format: html
date: 05-04-2025
image: Building_a_Company_Brochure_with_AI.jpg
---

This project demonstrates how to create a company brochure using AI. By leveraging web scraping and OpenAI's GPT-4o-mini, we can efficiently gather information from a company's website and generate compelling content for prospective clients, investors, and potential recruits. This approach streamlines the brochure creation process, making it faster and more cost-effective.

### Project Overview

The project involves scraping a company's website, identifying relevant links, and using the scraped content to automatically generate a brochure. We use Python for web scraping and data processing, and the OpenAI API to generate the brochure's content.

### Key Technologies

* **Python**: The primary programming language used for web scraping, data processing, and interacting with the OpenAI API.
* **Beautiful Soup**: A Python library for parsing HTML and XML documents, used here to extract information from web pages.
* **OpenAI API**: Used to generate the brochure content, leveraging the GPT-4o-mini model for its natural language processing capabilities.

### Implementation

#### 1. Importing Libraries

In [ ]:
# Import the libraries
import os # For interacting with the operating system, e.g., accessing environment variables.
import requests # For making HTTP requests to fetch webpage content.
import json # For working with JSON data, which is used for structuring the API responses.
from typing import List # For type hinting
from dotenv import load_dotenv # For loading environment variables from a .env file.
from bs4 import BeautifulSoup # For parsing HTML content, making it easier to extract specific data.
from IPython.display import display, Markdown # For displaying formatted text (Markdown) in the notebook.
from openai import OpenAI # For interacting with OpenAI's API.

#### 2. Loading API Key

We load the OpenAI API key from a .env file for security.

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Set OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI model
MODEL = 'gpt-4o-mini'
openai = OpenAI()

#### 3. Defining the `Website` Class

The `Website` class is defined to represent a website and its content. It fetches the HTML content of a given URL, extracts the text, and finds all the links.

In [7]:
# A class to represent the webpage scraper

# Some websites need you to use proper headers when fetching their content:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else 'No title found'
        if soup.body:
            for irrelevant in soup.body(['script', 'style', 'img', 'input']):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator='\n', strip=True)
        else:
            self.text = ''
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    
    def get_contents(self):
        return f"Webpage Title: {self.title}\nWebpage Content:\n{self.text}\n\n"


The class constructor does the following:
- Fetches the webpage content using requests.get().
- Parses the HTML using BeautifulSoup.
-  Extracts the title and text content, removing irrelevant elements like scripts and styles.
-  Extracts all the hyperlinks from the page.
- The get_contents() method returns the title and text of the webpage.


#### 4. Finding Relevant Links

The core of the project is identifying which links from the website are most relevant to include in the brochure. We use the gpt-4o-mini model to analyze the links and determine their relevance.

In [8]:
# Testing the Website class
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-

Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

In [11]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brouchure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    links: [
        {"type": "about_page", "url": "https://full.url/goes/here/about"},
        {"type": "careers_page", "url": "https://another.full.url/careers"},
}

"""

In [12]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brouchure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    links: [
        {"type": "about_page", "url": "https://full.url/goes/here/about"},
        {"type": "careers_page", "url": "https://another.full.url/careers"},
}




In [18]:
def get_links_user_prompt(website: Website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms and Service or Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

The `get_links_user_prompt` function generates the user prompt, providing the LLM with the list of links from the website and asking it to identify the relevant ones.

In [19]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms and Service or Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23

In [25]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object", }
    )

    result = response.choices[0].message.content
    return json.loads(result)

The `get_links` function takes a URL, creates a `Website` object, and uses the OpenAI API to get the relevant links. It sends the system and user prompts to the API, and then parses the JSON response.

In [21]:
# Testing 
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen3-235B-A22B',
 '/deepseek-ai/DeepSeek-Prover-V2-671B',
 '/nari-labs/Dia-1.6B',
 '/Qwen/Qwen3-30B-A3B',
 '/Qwen/Qwen3-32B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen3-Demo',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/stepfun-ai/Step1X-Edit',
 '/spaces/nvidia/describe-anything-model-demo',
 '/spaces',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/nvidia/Nemotron-CrossThink',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets/Eureka-Lab/PHYBench',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers',


In [26]:
get_links(huggingface.url)

{'links': [{'type': 'about_page', 'url': 'https://huggingface.co/'},
  {'type': 'enterprise_page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing_page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers_page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog_page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community_page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'github_page', 'url': 'https://github.com/huggingface'},
  {'type': 'linkedin_page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'twitter_page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [27]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)

    print(f"Found links: {links}")
    for link in links['links']:
        result += f"\n\n{link['type']}\n"
        result += Website(link['url']).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [29]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [30]:
def get_brochure_user_prompt(company_name: str, url: str):
    user_prompt = f"You are looking at a company called {company_name}.\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt += user_prompt[:7_000]  # Truncate if more than 7,000 characters
    return user_prompt

In [31]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about_page', 'url': 'https://huggingface.co'}, {'type': 'careers_page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise_page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog_page', 'url': 'https://huggingface.co/blog'}, {'type': 'models_page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets_page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces_page', 'url': 'https://huggingface.co/spaces'}, {'type': 'docs_page', 'url': 'https://huggingface.co/docs'}, {'type': 'community_page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github_page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter_page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin_page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called HuggingFace.\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title: Hugging Face – The AI community building the future.\nWebpage Content:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3-235B-A22B\nUpdated\n3 days ago\n•\n28.6k\n•\n644\ndeepseek-ai/DeepSeek-Prover-V2-671B\nUpdated\n4 days ago\n•\n1.71k\n•\n627\nnari-labs/Dia-1.6B\nUpdated\n7 days ago\n•\n108k\n•\n1.77k\nQwen/Qwen3-30B-A3B\nUpdated\n4 days ago\n•\n39.8k\n•\n428\nQwen/Qwen3-32B\nUpdated\n5 days ago\n•\n73k\n•\n263\nBrowse 1M+ models\nSpaces\nRunning\n5.86k\n5.86k\nDeepSite\n🐳\nGenerate any application w

#### 5. Generating the Brochure Content

The `create_brochure` function uses the gpt-4o-mini model to generate the brochure content. It takes the company name and URL as input, fetches the website content, identifies relevant links, and then prompts the LLM to create a brochure.

In [32]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

#### 6. Running the Example


In [33]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about_page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers_page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise_page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'posts_page', 'url': 'https://huggingface.co/posts'}, {'type': 'blog_page', 'url': 'https://huggingface.co/blog'}, {'type': 'community_page', 'url': 'https://discuss.huggingface.co'}, {'type': 'social_media_page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin_page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face

## The AI Community Building the Future

Hugging Face is a pioneering company dedicated to democratizing machine learning and artificial intelligence. With over **1 million models** and **250,000 datasets**, we provide a collaborative platform for researchers, developers, and enthusiasts in the machine learning community to share, innovate, and advance the state of AI technology.

---

### What We Offer

**Collaboration Platform**  
Join a global network where you can create, discover, and collaborate on machine learning models, datasets, and applications.

- **Models**: Explore the latest trending models like Qwen/Qwen3 and DeepSeek-Prover.
- **Datasets**: Access a rich repository of datasets for any ML task, including OpenMathReasoning and OpenCodeReasoning.
- **Spaces**: Experiment and run applications in a user-friendly interface, optimized for development.

---

### Why Choose Hugging Face?

- **Community-Driven**: Actively engage with more than **50,000 organizations** including industry giants like **Google**, **Amazon**, and **Microsoft** who trust our platform.

- **Open Source**: We believe in transparency and availability. Contribute to our models and explore extensive resources including Transformers, Tokenizers, and more.

- **Enterprise Solutions**: With specialized features for organizations, Hugging Face provides security, access controls, and prioritized support to help your team effectively build AI applications.

---

### Company Culture

We are on a mission to **democratize good machine learning**, making it accessible for everyone involved in the AI ecosystem. Our community is composed of **over 200 dedicated employees** who continually contribute to fostering an inclusive and innovative work environment. 

We encourage continuous learning, collaboration, and creativity, allowing our team members to thrive and push the boundaries of what's possible in AI.

---

### Career Opportunities

We're constantly looking for talented individuals to join our dynamic team. If you're passionate about **AI**, **machine learning**, or **software development**, consider exploring open positions with us. Dive into the world of cutting-edge AI technology and help us shape the future with your expertise.

- **Explore Open Positions**: Join our collaborative work environment that values innovation and learning.

---

### Join Us!

Whether you're looking to leverage our powerful tools in your projects, become a part of our thriving community, or start a career with us, Hugging Face is the place to be. 

**Contact Us | [Visit Our Website](https://huggingface.co)**

_huggingface - The AI community building the future._

In [35]:
create_brochure("Anthropic", "https://www.anthropic.com/")

Found links: {'links': [{'type': 'about_page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers_page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research_page', 'url': 'https://www.anthropic.com/research'}, {'type': 'team_page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news_page', 'url': 'https://www.anthropic.com/news'}]}


```markdown
# Anthropic Brochure

## About Anthropic
At Anthropic, our mission is clear: **to build AI systems that serve humanity's long-term well-being**. We are a Public Benefit Corporation headquartered in San Francisco, dedicated to creating reliable, interpretable, and steerable AI systems. Our pioneering work not only focuses on powerful technologies, but also emphasizes the importance of considering their societal implications through rigorous research and policy-making.

## Our Vision
We believe AI will have a vast impact on the world. By harnessing cutting-edge research and applying our findings to create tangible products, we aim to ensure that AI serves as a positive force for humanity. Our flagship model, **Claude**, represents our commitment to developing advanced AI tools that are safe, interpretable, and beneficial for all.

## Company Culture
### Values
Our organization thrives on a unique set of values:
1. **Act for the Global Good**: We maximize positive outcomes for humanity while being bold in our actions.
2. **Hold Light and Shade**: We acknowledge the risks and rewards of AI, striving to protect against the bad while amplifying the good.
3. **Be Good to Our Users**: We extend kindness and generosity to everyone influenced by our technology, including customers, policy-makers, and employees.
4. **Ignite a Race to the Top on Safety**: We aim to inspire all developers to prioritize safety in AI systems, setting high standards for the industry.
5. **Do the Simple Thing That Works**: We embrace practical, effective solutions that have a meaningful impact.
6. **Be Helpful, Honest, and Harmless**: Our high-trust culture emphasizes integrity, collaboration, and a shared mission.
7. **Put the Mission First**: Our mission guides our decisions and actions, fostering collaboration and trust across teams.

## Research and Development
Our interdisciplinary team comprises researchers, engineers, policy experts, and operational leaders, all collaborating to push the boundaries of AI safety. Our research spans various modalities, tackling novel areas such as interpretability, human feedback, and societal impact analysis.

## Customer Solutions
We translate our research into practical tools and applications that benefit various sectors, including:
- **AI Agents** for automated tasks
- **Coding support** to assist developers
- **Customer service solutions** for improved interaction
- **Specialized applications** for education and enterprise needs

## Career Opportunities
### Join Us!
At Anthropic, we value a diverse range of experiences and ideas. Our team members come from backgrounds in physics, machine learning, public policy, and business. We are committed to creating a supportive work environment where everyone can thrive. Some of our employee benefits include:
- Comprehensive health, dental, and vision insurance
- Flexible paid time off and paid parental leave
- Retirement plans with competitive matching
- Wellness stipends and commuter benefits

If you're passionate about AI safety and want to make a real difference, [**join our team**](#careers).

## Customer Testimonials
> "Claude has transformed our way of working at North Highland. It has enabled us to complete tasks up to 5x faster!"  
> — Luka Anic, Senior Director at North Highland

> "Leveraging Claude in our workflows has made generating content and insights significantly easier and faster."  
> — Olga Pirog, Head of AI Transformation at IG Group

## Stay Updated
For the latest news and developments in AI safety and our product offerings, visit our [**Newsroom**](#news) and follow us on our social media channels.

---

### Contact Us
For inquiries, partnerships, and support, please visit our website at [Anthropic.ai](https://www.anthropic.ai).

---

**Anthropic**: Crafting the Future of Safe AI.
```

In [36]:
create_brochure("Maruti Suzuki", "https://www.marutisuzuki.com/")

Found links: {'links': [{'type': 'about_page', 'url': 'https://www.marutisuzuki.com/corporate/about-us'}, {'type': 'about_history', 'url': 'https://www.marutisuzuki.com/corporate/about-us/history'}, {'type': 'about_leadership', 'url': 'https://www.marutisuzuki.com/corporate/about-us/leadership'}, {'type': 'about_strength', 'url': 'https://www.marutisuzuki.com/corporate/about-us/strength'}, {'type': 'about_values', 'url': 'https://www.marutisuzuki.com/corporate/about-us/values'}, {'type': 'about_exports', 'url': 'https://www.marutisuzuki.com/corporate/about-us/exports'}, {'type': 'about_sustainability', 'url': 'https://www.marutisuzuki.com/corporate/about-us/sustainability'}, {'type': 'about_csr', 'url': 'https://www.marutisuzuki.com/corporate/about-us/csr'}, {'type': 'careers_page', 'url': 'https://www.marutisuzuki.com/corporate/careers'}, {'type': 'careers_life', 'url': 'https://www.marutisuzuki.com/corporate/careers/life-at-msil'}, {'type': 'careers_why_work_with_us', 'url': 'https:/

# Maruti Suzuki: Driving India Forward

## Overview
Maruti Suzuki India Limited (MSIL) is the largest passenger vehicle manufacturer in India, recognized for its commitment to quality and reliability. Celebrated for manufacturing a wide array of cars, including hatchbacks, sedans, SUVs, and vans, Maruti Suzuki’s brands include ARENA, NEXA, True Value, and Commercial.

## Company Culture
At Maruti Suzuki, the work culture is rooted in teamwork, innovation, and continuous learning. The organization fosters an atmosphere that embraces diversity, encourages openness, and values employee contributions, ensuring that every voice is heard. Employees enjoy supportive policies aimed at professional growth and personal development, making Maruti Suzuki not just a workplace, but a vibrant community.

### Why Work with Us?
- **Career Growth**: We offer extensive training and mentorship programs to help you maximize your potential.
- **Employee Welfare**: Benefits like subsidized meals, flexible work hours, and wellness programs ensure a healthy work-life balance.
- **Recognition**: Exceptional performance is acknowledged and rewarded, fostering a culture of excellence.

## Commitment to Sustainability
Maruti Suzuki prioritizes environmental responsibility. The company invests in green technologies, striving to reduce its ecological footprint by developing eco-friendly vehicles and managing resources efficiently. The implementation of best practices in sustainability ensures Maruti Suzuki is not just a leader in manufacturing but also in eco-friendliness.

## Corporate Social Responsibility (CSR)
Maruti Suzuki is dedicated to uplifting the communities it serves through various initiatives in education, health, and environmental conservation. The firm values corporate citizenship and actively engages in philanthropic efforts to promote societal well-being.

## Come Join Us
Maruti Suzuki is always on the lookout for diverse talent to join its expanding team. Whether you are a fresh graduate, a seasoned professional, or looking for workmen hiring (ITI), Maruti Suzuki offers many opportunities across various functions and locations.

- **Current Openings**: Check out the careers page for fresh engineering graduates, experienced professionals, and apprentices.
- **Training Academy**: Our in-house training program equips employees with the skills needed for success in the automotive industry.

## Innovation and Technology
With a strong focus on R&D, Maruti Suzuki leverages cutting-edge technology to enhance vehicle performance and efficiency. The company is committed to advancements in hybrid, Smart Hybrid, and CNG technology.

## Contact Us
For inquiries or to learn more about our offerings, please visit our website or reach us at:
- **Phone**: 1800 102 1800
- **Email**: contact@maruti.co.in

Join Maruti Suzuki in shaping the future of mobility in India. Drive change, drive innovation, drive with Maruti Suzuki!